Import the libraries

In [5]:
from dotenv import load_dotenv
from langchain import hub
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
import os
import datetime
import urllib
from openai import OpenAI
import chromadb

load_dotenv()

True

Define tools

In [10]:
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from test.test_typing import Annotated
import smtplib

embedding_client = OpenAI(api_key="fake", base_url='http://127.0.0.1:1234/v1')
chroma_client = chromadb.PersistentClient(path="./chroma_db")  
collection = chroma_client.get_or_create_collection(name="movies")

def generate_prompt(prompt_path: str, values: list[str]):
    with open(prompt_path, 'r') as file:
        data = file.read().rstrip()
        index=0
            
        while(data.find(f"!<INPUT_{index}>") > 0):
            data = data.replace(f"!<INPUT_{index}>", values[index] if len(values) > index is not None else '')
            index+=1
        return data

    return None;

def get_embedding(text):
    response = embedding_client.embeddings.create(model="text-embedding-nomic-embed-text-v1.5", input=text)
    return response.data[0].embedding

@tool("get_current_datetime")
def get_current_datetime() -> str :
    """Get current date and time"""
    return datetime.datetime.now().strftime('%m/%d/%Y, %H:%M:%S')

@tool("convert_to_japanse_datetime_custom")
def convert_to_japanse_datetime(date_str: Annotated[str, 'date string in ISO format']) -> str :
    """Convert date string to custom japanese date time"""
    dt = datetime.datetime.fromisoformat(date_str)
    return f"{dt.year}年{dt.month}月{dt.day}日　{dt.hour}時{dt.minute}分{dt.second}秒"

# dir: Annotated[str, 'the directory of the file location, pass in /tmp for default']
@tool("download_image_from_url")
def download_image_from_url(url: Annotated[str, 'full URL of the web image']) -> dict:
    "Download an image from web and save to local device"
    directory = f'{os.getcwd()}/tmp'
    file_name = f"{directory}/{datetime.datetime.timestamp(datetime.datetime.now())}-agent.png"
    cleaned_url = url.replace("'", '')
    try:
        res= urllib.request.urlretrieve(cleaned_url,file_name)    
        print(res)
        os.startfile(file_name)
        return  {
            "success": True,
            "file_name": file_name
        }
    except (Exception) as e:
         return {
                "success": False,
                "error": f"{e}",
                "url": url
            }
         
@tool("send_email")
def send_email(to: Annotated[str, 'the receiver\'s email address'], name: Annotated[str, 'the receiver\'s name'], content: Annotated[str, 'the content of the email'], attachment_paths: Annotated[list[str], 'full paths of the files to be attached to the mail, default to an empty list']) -> dict :
    """Send an email to specified email address, with name, content and optional attachments"""
    msg = MIMEMultipart()
    msg['Subject'] = 'Generated By AI Agent'
    msg['From'] = os.getenv('SMTP_USERNAME')
    msg['To'] = to
    msg.attach(MIMEText(f'[Powered by Benihayashi\'s AI Agent] \n{content}', "plain"))
    print(msg)
    try:
        server = smtplib.SMTP(os.getenv('SMTP_HOST'),587)
        server.starttls() 
        server.login(os.getenv('SMTP_USERNAME'), os.getenv('SMTP_PASSWORD'))
        print(msg['From'], msg['To'], msg.as_string())
        server.sendmail(msg['From'], msg['To'], msg.as_string())
        server.quit()
        return {
            "success": True,
            "subject": msg['Subject'],
        }
    except Exception as e:
        print(e)
        return {
            "success": False,
            "error": e,
        }
        

@tool("search_movies")
def search_movies(query: Annotated[str, 'optimized query to search for movie from chromadb'], n_results: Annotated[int, 'number of results']) -> dict :
    """Search for a movie from a vector database"""
    query_embedding = get_embedding(query)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=n_results
    )
    return '\n\n'.join(results["documents"][0])

@tool("summarize_movies")
def summarize_movies(movie_descriptions: Annotated[list[str], 'a list of movies and its descriptions'],) -> str :
    """Takes in an array of raw movie name and description and generate summary of the movies"""
    llm = ChatOpenAI(
            model="gpt-4o-mini",
            api_key=os.getenv('OPENAI_KEY'),
        )
    prompt = generate_prompt('./prompts/movie_summary.txt', '\n\n'.join(movie_descriptions))
    output = llm.invoke([HumanMessage(content=prompt)])
    return output.text()

    

Initialized OpenAI LLM

In [11]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Initialize a ChatOpenAI model
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.getenv('OPENAI_KEY'),
)

tools = [
    get_current_datetime,
    convert_to_japanse_datetime,
    download_image_from_url,
    send_email,
    search_movies,
    summarize_movies,
]

# prompt = hub.pull("hwchase17/react",api_key=os.getenv('LANGSMITH_KEY'),)
memory = MemorySaver()
agent = create_react_agent(
    llm,
    tools=tools,
    # prompt=prompt,
    checkpointer=memory,
)


Testing

In [12]:
import uuid

config = {"configurable": {"thread_id": uuid.uuid4()}}
#prompt1 = "Download this image for me: https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS4ZNGq9_b4Z0Gw-_HXeJLlI0wOPUJnT020Xg&s"
# prompt1= "Send an email to bennylim0403@gmail.com, his name is Benny Lim, give him a simple greetings"
prompt1= "Recommend some japanese romantic movies for me, 4 will be enough"
response = agent.invoke({"messages": [HumanMessage(prompt1)]},config=config)
print("question: ", prompt1)
print("response: ", response['messages'][-1].content)
# for event in agent_executor.stream({"input": [HumanMessage(content=prompt1)]}, config):
#     print("question: ", prompt1)
#     print("response: ", event['output'])

Add of existing embedding ID: Big Brother-None
Add of existing embedding ID: Big Brother-None
Add of existing embedding ID: Big Brother-None
Add of existing embedding ID: Big Night Out-None
Add of existing embedding ID: Binnenstebuiten-None
Add of existing embedding ID: Black Dog-None
Add of existing embedding ID: Blanda drops-None
Add of existing embedding ID: Blank Slate-None
Add of existing embedding ID: Blankety Blank-None
Add of existing embedding ID: Blind Date-None
Add of existing embedding ID: Blind Date-None
Add of existing embedding ID: Blind Date-None


question:  Recommend some japanese romantic movies for me, 4 will be enough
response:  Here are some Japanese romantic movies you might enjoy:

1. **Bakumatsu**
   - **Description**: In Kyoto, a heroine meets a man who gives her a pocket watch, leading her to a Torii in a shrine, where she is warped back into the Bakumatsu period. In this world of differing values, she tries to find a way back while a love story blossoms with a man who has crossed time.
   - **Genre**: Romantic Fantasy

2. **99 Years of Love - Japanese Americans**
   - **Description**: This film follows a family of Japanese immigrants in America over 99 years, focusing on their struggles during the war, racism, and family dynamics. It explores the love and sacrifices of family members separated by war.
   - **Genre**: Drama, War

3. **Asatte no Houkou**
   - **Description**: A young girl and a woman, tied by their secret desires, navigate life after their parents' death and a lost love. Their lives intertwine on a hot 

In [13]:
prompt2 = "can summarise these movies and send to bennylim0403@gmail.com?"
response = agent.invoke({"messages": [HumanMessage(prompt2)]},config=config)
print("question: ", prompt2)
print("response: ", response['messages'][-1].content)

Content-Type: multipart/mixed; boundary="===============6033489324708543909=="
MIME-Version: 1.0
Subject: Generated By AI Agent
From: benny.lim@tresoftsys.com
To: bennylim0403@gmail.com

--===============6033489324708543909==
Content-Type: text/plain; charset="us-ascii"
MIME-Version: 1.0
Content-Transfer-Encoding: 7bit

[Powered by Benihayashi's AI Agent] 
Here are the summaries of the recommended Japanese romantic movies:

1. **Bakumatsu**: In Kyoto, a heroine meets a man who gives her a pocket watch, leading her to a Torii in a shrine, where she is warped back into the Bakumatsu period. In this world of differing values, she tries to find a way back while a love story blossoms with a man who has crossed time.

2. **99 Years of Love - Japanese Americans**: This film follows a family of Japanese immigrants in America over 99 years, focusing on their struggles during the war, racism, and family dynamics. It explores the love and sacrifices of family members separated by war.

3. **Asatt